In [1]:
import wikipedia
wikipedia.set_lang('simple')
dt = wikipedia.page("Donald Trump")
text = dt.summary

In [31]:
text = """
This is the most weird day that I have ever had . I had been walking to my friend's house when I saw a bull in the street!. In that moment, I ran too fast to the opposite direcion meanwhile I was screaming for help. I had experienced a similar situation like that, but in a secure environment .

I have a mix of feelings now. I am thinking about before, I have been running too fast, I thought that the bull was near me after that race.

The next year I will have been writing in this diary for 10 years, and this is the most weird fact that I have ever had.

Now, what I want is: talk with my friend face-to-face about that moment, so, I think that I will go to his house tomorrow well the path, because I could see other one again!)

I will close this crazy day with the hope that for the next year I will not have lived another day like this.

I could have tried and should have tried it.
"""

In [ ]:
def get_greedy_matches(matches):
    
    intervals = [[match[1],match[2]] for match in matches]
    
    sorted_by_lower_bound = sorted(intervals, key=lambda tup: tup[0])
    merged = []

    for higher in sorted_by_lower_bound:
        if not merged:
            merged.append(higher)
        else:
            lower = merged[-1]
            # test for intersection between lower and higher:
            # we know via sorting that lower[0] <= higher[0]
            if higher[0] <= lower[1]:
                upper_bound = max(lower[1], higher[1])
                merged[-1] = [lower[0], upper_bound]  # replace by merged interval
            else:
                merged.append(higher)
    indices = []
    errors = 0
    error_merges = []
    
    for merge in merged:
        try:
            indices.append(intervals.index(merge))
        except:
            errors += 1
            error_merges.append(merge)
            
    print(str(errors) + "errors occured. The erroroneous merge is " + str(error_merges))
    
    new_matches = [matches[index] for index in indices]
    
    return new_matches

In [ ]:
'''
https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
Tenses: 
Present: VBP/VBZ
Present continuous: am/is/are/'m/'s/'re + (RB) + VG
Past: VBD
Past continuous: was/were + (RB) + VG
Future: will/wo/shall + (RB) + VB
Future II: am/is/are/'m/'s/'re + (RB) + going + to + VB
Future continuous: will/wo/'ll + (RB) + be + (RB) + VG
Present perfect: have/has/'ve' + (RB) + VBN
Present perfect continuous: have/has + (RB) + been + (RB) + VG
Past perfect: had + (RB) + VBN
Past perfect continuous: had + (RB) + been + (RB) + VG
Future perfect: will/wo/'ll + (RB) + have + (RB) + VBN
Future perfect continuous: will/wo + (RB) + been + (RB) + VG
Modal verbs: can/could/may/might/should/must/would/'d + (RB) + VB + (RB) + V(any)
(Actually include future tenses)
(*need to handle tenses myself)
'''

In [32]:
import spacy
from spacy.matcher import Matcher
import re

# Remove parentheses and the contents contained in them
text = re.sub(r'\([^)]*\)', '',text)
nlp=spacy.load("en_core_web_sm")

tenses_priority = ["future_perfect_continuous","past_perfect_continuous",
                  "present_perfect_continuous", "future_perfect", "past_perfect",
                  "present_perfect", "future_continuous", "past_continuous",
                  "present_continuous", "future_tense", "past_tense", "present_tense", '']

is_list = ["is","am","are","\'s","\'re","ain\'t"]
do_list = ["do", "does"]
was_list = ["was","were"]
will_list = ["will","wo","shall"]
have_list = ["have","\'ve","has","\'s"]
adv_dict = {"TAG":"RB","OP":"*"}

matcher = Matcher(nlp.vocab)
present_tense_pattern = [[{"TAG":{"IN":["VBP","VBZ"]}},adv_dict,{"TAG":"VB", "OP":"*"}]]
past_tense_pattern = [[{"LOWER":"did"},adv_dict,{"TAG":"VB"}],
                    [{"TAG":"VBD"}],
                     [{"LOWER":{"IN":was_list+["did"]}},adv_dict,{"TAG":"PRP"},adv_dict]]
future_tense_pattern = [[{"LOWER":{"IN":will_list}},adv_dict,{"TAG":"VB"}]]
present_cont_pattern = [[{"LOWER":{"IN":is_list}},adv_dict,{"TAG":"VBG"}]]
past_cont_pattern = [[{"LOWER":{"IN":was_list}},adv_dict,{"TAG":"VBG"}]]
future_cont_pattern = [[{"LOWER":{"IN":will_list}},adv_dict,
                       {"LOWER":"be"},adv_dict,{"TAG":"VBG"}]]
present_perfect_pattern = [[{"LOWER":{"IN":have_list}},
                            adv_dict,{"TAG":"VBN"}]]
past_perfect_pattern = [[{"LOWER":"had"},adv_dict,{"TAG":"VBN"}]]
future_perfect_pattern = [[{"LOWER":{"IN":will_list}},adv_dict,
                          {"LOWER":{"IN":have_list}},adv_dict,{"TAG":"VBN"}]]
present_perfect_cont_pattern = [[{"LOWER":{"IN":have_list}},adv_dict,
                                 {"LOWER":"been"},adv_dict,
                                 {"TAG":"VBG"}]]
past_perfect_cont_pattern = [[{"LOWER":"had"},adv_dict,{"LOWER":"been"},
                              adv_dict,{"TAG":"VBG"}]]
future_perfect_cont_pattern = [[{"LOWER":{"IN":will_list}},adv_dict,{"LOWER":"have"},
                                adv_dict,{"LOWER":"been"},adv_dict,
                                {"TAG":"VBG"}]]
modal_present_pattern = [[
    
]]


doc = nlp(text)
matcher.add("present_tense", present_tense_pattern, greedy='LONGEST')
# matcher.add("present_tense_question", present_tense_q)
matcher.add("past_tense", past_tense_pattern, greedy='LONGEST')
matcher.add("future_tense", future_tense_pattern)
matcher.add("present_continuous", present_cont_pattern)
matcher.add("past_continuous", past_cont_pattern)
matcher.add("future_continuous", future_cont_pattern)
matcher.add("present_perfect", present_perfect_pattern)
matcher.add("past_perfect", past_perfect_pattern)
matcher.add("future_perfect", future_perfect_pattern)
matcher.add("present_perfect_continuous", present_perfect_cont_pattern)
matcher.add("past_perfect_continuous", past_perfect_cont_pattern)
matcher.add("future_perfect_continuous", future_perfect_cont_pattern)

# replace_word(text, "__________")

matches = matcher(doc)
matches.sort(key=lambda x:x[1])

print("Original matches:")
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(string_id, start, end, span.text)
            
print("\n")
        
matches = get_greedy_matches(matches)

print("New matches:")
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(string_id, start, end, span.text)
            
    


Original matches:
present_tense 2 3 is
present_perfect 9 12 have ever had
present_tense 9 11 have ever
past_perfect 14 16 had been
past_perfect_continuous 14 17 had been walking
past_tense 14 15 had
past_tense 24 25 saw
past_tense 37 38 ran
past_continuous 46 48 was screaming
past_tense 46 47 was
past_perfect 52 54 had experienced
past_tense 52 53 had
present_tense 68 69 have
present_continuous 76 78 am thinking
present_tense 76 77 am
present_perfect 82 84 have been
present_perfect_continuous 82 85 have been running
present_tense 82 83 have
past_tense 89 90 thought
past_tense 93 94 was
future_tense 105 107 will have
future_perfect 105 108 will have been
future_perfect_continuous 105 109 will have been writing
present_perfect 106 108 have been
present_perfect_continuous 106 109 have been writing
present_tense 118 119 is
present_perfect 125 128 have ever had
present_tense 125 127 have ever
present_tense 134 135 want
present_tense 135 136 is
present_tense 153 154 think
future_tense 156 15

This is the most weird day that I have ever had . I had been walking  to my friend's house when  I saw  a bull in the street!. In that moment, I ran too fast to the opposite direcion meanwhile I was screaming  for help. I had experienced  a similar situation like that, but in a secure environment .

I have  a mix of feelings now. I am thinking  about before, I have been running  too fast, I thought that the bull was near me after that race.

The next year I will have been writting  in this diary for 10 years, and this is the most weird fact that I have ever had.

Now, what I want is: talk with my friend face-to-face about that moment, so, I think that I will go  to his house tomorrow  well the path, because I could see other one again!)

I will close this crazy day with the hope that for the next year I will not have lived  another day like this.

In [17]:
pattern = [[{"TAG":{"IN":['VBZ','VBP']}},adv_dict,{"POS":"VERB","OP":"!"}]]

text = "I have not done yet."
doc = nlp(text)
matcher2 = Matcher(nlp.vocab)
matcher2.add("pattern",pattern)
matches = matcher2(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(string_id, start, end, span.text)

In [35]:
doc = nlp("I won't do it.")

for token in doc:
    # Get the token text, part-of-speech tag and dependency label
    token_text = token.text
    token_pos = token.pos_
    token_tag = token.tag_
    token_dep = token.dep_
    token_morph = token.morph
    # This is for formatting only
    print(f"{token_text:<12}{token_pos:<10}{token_tag:<10}{token_dep:<10}")
    print(token_morph)

I           PRON      PRP       nsubj     
Case=Nom|Number=Sing|Person=1|PronType=Prs
wo          AUX       MD        aux       
VerbType=Mod
n't         PART      RB        neg       

do          VERB      VB        ROOT      
VerbForm=Inf
it          PRON      PRP       dobj      
Case=Acc|Gender=Neut|Number=Sing|Person=3|PronType=Prs
.           PUNCT     .         punct     
PunctType=Peri


In [109]:
spacy.explain("VBN")

'verb, past participle'